In [317]:
import pandas as pd 
import requests
import json

In [318]:
pd.set_option('display.max_colwidth', None)

In [319]:
COW_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/cowprotocol/cow"
COW_AUCTION_API = "https://api.cow.fi/mainnet/api/v1/solver_competition/by_tx_hash/"

def make_graph_query_request(_query):
    return requests.post(COW_GRAPH_URL, json={'query': _query}).json()

def get_cow_auction(tx_hash):
    return requests.get(COW_AUCTION_API + tx_hash).json()

In [320]:
opps = pd.json_normalize(json.load(open('data/opportunities.json')))
len(opps["cowOrder.id"].unique())

79

In [321]:
query = """
    {
        orders(where: {
            id_in: [%s]
        }) {
            id
            trades(first: 10) {
                timestamp, 
                txHash, 
                settlement {
                    solver { address }
                }
                sellToken {
                    decimals
                }
                buyToken {
                    decimals
                }
                sellAmount	
                buyAmount
                feeAmount
            }
        }
    }""" % ",".join([f'"{i}"' for i in opps["cowOrder.id"].unique()])

graph_response = make_graph_query_request(query)

In [322]:
settlements = []

for order in graph_response["data"]["orders"]:
    for trade in order["trades"]:
        auction = get_cow_auction(trade["txHash"])
        winning_surplus = 0 if auction.get("solutions") is None else max([s["objective"]["surplus"] for s in auction["solutions"]])

        settlements.append({
            "order_id": order["id"],
            "solver": trade["settlement"]["solver"]["address"],
            "sell_amount_fix": (int(trade["sellAmount"])-int(trade["feeAmount"])) / 10**trade["sellToken"]["decimals"],
            "buy_amount_fix": int(trade["buyAmount"]) / 10**trade["buyToken"]["decimals"],
            "winning_surplus": winning_surplus,
            "tx_hash": trade["txHash"],
        })

settlements_df = pd.DataFrame(settlements)
settlements_df

,order_id,solver,sell_amount_fix,buy_amount_fix,winning_surplus,tx_hash
0,0x0235fdf6659eaa3d6273f32972f866bfba297baf7b67d24851566568e8499ad466ea2d335291a5afa8d3522eef6adfc3f8746e16643d4d90,0xc9ec550bea1c64d779124b23a26292cc223327b6,11818.082972,5.703457,2.620104e+16,0x4510799898c7e60d1106b5a5d0458c112a701d67425d45f33678e989ec074f22
1,0x05f7ddbdf798e6baa07342983f1e315a498e46a0953ff3c6e5a9c8e1c7bdef550b5dd26f44d36141d97a3bf6841e7ba63c092af9643d94ab,0x3cee8c7d9b5c8f225a8c36e7d3514e1860309651,1586.443392,0.761862,2.684053e+15,0xfada5e97917a391192314a2df1aeda43fcce8f1b6b37da431a9b6e8064f27363
2,0x0b77e604f4791a37605d2e3b807e3dd0dfbe7d8ac27cdedb88797c4c0480ac2989ecb99d52f9c0c7d3aa574178278bcc3276c965643d7501,0x3cee8c7d9b5c8f225a8c36e7d3514e1860309651,483.835431,0.233405,1.769505e+15,0x895312eaa300339c2e96e62e6143daff9d7db62143390d253dd7f80f3168d8f4
3,0x0c5e2371e319c65ded98ec29a238d43386eacd3291af8ad882bbaf37f146267440a50cf069e992aa4536211b23f286ef88752187ffffffff,0xa21740833858985e4d801533a808786d3647fb83,12.195676,24081.454203,2.275725e+17,0x84197a1b7560c9fd4a4728d67e491f320693e6eb97844cfe03cd64ccb71170af
4,0x0de23176a4cb67a88bbe54674648ee03e1a853e8ca80eef1b1b416284df163a589ecb99d52f9c0c7d3aa574178278bcc3276c965643ecdaa,0x149d0f9282333681ee41d30589824b2798e9fb47,1966.293645,0.937974,1.362209e+16,0xc2cc712a2bf73824c08858f0264fb97a8f691c77940c8eeacc9127d5b5c14f65
...,...,...,...,...,...,...
65,0xe22b5b5eda2986bb265228b7fda0ea4f6bf9fbd9b450aeb1eb99d844d915cee840a50cf069e992aa4536211b23f286ef88752187ffffffff,0x149d0f9282333681ee41d30589824b2798e9fb47,449.971836,891002.326578,9.405283e+18,0x5bd894cf74dac9f8341c52580aedfb9b741ad3e89d047b00bc577f0fac7f74aa
66,0xee74215969daee5a04c50ef24b15c843e23bf92c32764e44e7a2a66f7b874bc7719304183f09a84c4d8da8d64bdb6c6e0eeede4e643e8124,0xa21740833858985e4d801533a808786d3647fb83,3983.468075,1.892687,1.129446e+17,0x5c98c4d1c4a40886ff79838f6e598d6aeeda86df7ee69a2d4d77be8ae045885f
67,0xf5db176140ae2bdcc02277e8365f6e5078ded4d1cbfbc07f1c58970dcd0ae65776041b19713f5a0481b24a8bc2d90960bac18989643d8bbe,0x149d0f9282333681ee41d30589824b2798e9fb47,3969.368945,1.907634,3.294935e+15,0xd136ddecda6e39314bb1baf1e968a91330e41c24a58eef3e02bbbcdefde64731
68,0xf9a48c7d293928963710f2c4d3f430a96f9c57807090b36132a7595cdedd5c9040a50cf069e992aa4536211b23f286ef88752187ffffffff,0xc9ec550bea1c64d779124b23a26292cc223327b6,1.830845,3880.372552,3.671450e+16,0xa1cda42d65e95eef7d57fa7a4b436364089509f9372d11b93142e4f990e30d0f


### Positive spread

In [323]:
def get_duration_sec(series):
    return int((series.max()-series.min()) / 1e3)



opps_with_settlements = opps.merge(settlements_df, left_on="cowOrder.id", right_on="order_id", how="inner")
prices = opps_with_settlements['binanceOrderSpreadEth']/(opps_with_settlements["cowOrder.amountIn"] - opps_with_settlements["binanceMatch.amountIn"] + opps_with_settlements["binanceMatch.amountOut"] - opps_with_settlements["cowOrder.amountOut"])
opps_with_settlements["spread_binance_settlement"] = ((opps_with_settlements["sell_amount_fix"] - opps_with_settlements["binanceMatch.amountIn"]) + (opps_with_settlements["binanceMatch.amountOut"] - opps_with_settlements["buy_amount_fix"]))*prices
opps_with_settlements["spread_binance_order"] = opps_with_settlements["binanceOrderSpreadEth"]
opps_with_settlements["spread_binance_oneinch"] = opps_with_settlements["binanceOneinchSpreadEth"]
opps_with_settlements["winning_surplus"] /= 1e18

# groupby order-id and agg: max spread, min spread, count, 
agg_opps = opps_with_settlements \
    .groupby(by="cowOrder.id") \
    .agg(
        spread_binance_settlement_max=("spread_binance_settlement", max),
        spread_binance_settlement_min=("spread_binance_settlement", min),
        spread_binance_order_max=("spread_binance_order", max),
        spread_binance_order_min=("spread_binance_order", min),
        spread_binance_oneinch_max=("spread_binance_oneinch", max),
        spread_binance_oneinch_min=("spread_binance_oneinch", min),
        duration_sec=("timestamp_ms", get_duration_sec),
        winning_surplus=("winning_surplus", max),
    ) \
    .reset_index() \
    .sort_values(by="spread_binance_settlement_max", ascending=False)
agg_opps["have_winning_surplus"] = agg_opps["spread_binance_order_max"] > agg_opps["winning_surplus"]

agg_opps



,cowOrder.id,spread_binance_settlement_max,spread_binance_settlement_min,spread_binance_order_max,spread_binance_order_min,spread_binance_oneinch_max,spread_binance_oneinch_min,duration_sec,winning_surplus,have_winning_surplus
65,0xe22b5b5eda2986bb265228b7fda0ea4f6bf9fbd9b450aeb1eb99d844d915cee840a50cf069e992aa4536211b23f286ef88752187ffffffff,0.104753,-0.065149,9.510036,9.340135,855.220063,518.662397,7,9.405283,True
50,0x9c821bba240731bf9cf6e01529fd892a75e370fefb0d70104cfcbd5272517de4f6f6d531ed0f7fa18cae2c73b21aa853c765c4d864481fad,0.062861,0.028831,0.363194,0.329164,0.069333,0.035302,9,0.311894,True
60,0xc4053191dbcbc6d6d91441e5dc2e312db15a67db23af9308a823a36596d866a9fbe87d602f7d7dd511349be4acf58842392124a064444d84,0.055752,-0.001465,0.058557,0.001340,0.074031,-0.013649,275,0.002805,True
38,0x7a930699d5e9bb1e0a19ad387049e9602de54b349597264a5ab342eb5ba71819b9e025a1363373e48da72f7e4f6eb7cddf2f6501643fec1c,0.055740,-0.007505,0.604859,0.541614,0.067136,0.003228,24,0.631320,False
27,0x5c1b33929dbd24d765cf4355e56cd021180541089cb0a3843ebaefa8065e1fe5f6f6d531ed0f7fa18cae2c73b21aa853c765c4d8643eaaf3,0.037936,0.029518,0.086087,0.077669,79.777381,62.071316,7,0.067479,True
...,...,...,...,...,...,...,...,...,...,...
51,0xa4047c8eb429f92206696b970cde792b04acea099ac44ce57dfbd7991915cf4040a50cf069e992aa4536211b23f286ef88752187ffffffff,-0.004644,-0.004922,0.005103,0.004819,0.715541,0.130291,45,0.009741,False
68,0xf9a48c7d293928963710f2c4d3f430a96f9c57807090b36132a7595cdedd5c9040a50cf069e992aa4536211b23f286ef88752187ffffffff,-0.005117,-0.005766,0.031598,0.030949,-1.379617,-2.752751,6,0.036715,False
34,0x6c872d0c5054d33cb6d7861041c73edafd7444ca52522adc2da62f9d6a215d3c40a50cf069e992aa4536211b23f286ef88752187ffffffff,-0.009046,-0.026425,1.193267,1.175888,40.641001,6.188157,3,1.202313,False
26,0x55f48c5f0df7fb2a82cad4f3cfc40ec546d1fddebe8f30fc619bdcd5dd97dd87cdefed59e93d90b5b2cc6ac4231d12ee38b69a9b6447d546,-0.017439,-0.023177,0.005781,0.000043,0.005567,-0.000695,7244,0.023220,False


In [324]:
agg_opps.loc[(agg_opps.spread_binance_settlement_max > 0) & (agg_opps.have_winning_surplus)]["spread_binance_settlement_max"].sum()

0.34676349513482013

## Simulation 

In [325]:
from collections import defaultdict


best_opps_for_order = opps_with_settlements.loc[opps_with_settlements.groupby("cowOrder.id")["spread_binance_order"].idxmax(axis=0)]
best_opps_for_order.loc[best_opps_for_order.tokenIn == "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee", "tokenIn"] = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
best_opps_for_order.loc[best_opps_for_order.tokenOut == "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee", "tokenOut"] = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"


starting_balance = 1e6
binance_balance = defaultdict(lambda: starting_balance)
binance_balance_min = defaultdict(lambda: starting_balance)
onchain_balance = defaultdict(lambda: starting_balance)
onchain_balance_min = defaultdict(lambda: starting_balance)

for (i, order) in best_opps_for_order.sort_values(by="timestamp_ms").iterrows():
    binance_amount_in = order["binanceMatch.amountIn"]
    binance_amount_out = order["binanceMatch.amountOut"]
    token_in = order["tokenIn"]
    token_out = order["tokenOut"]

    if binance_balance[token_in] < binance_amount_in:
        print(f"Insufficient balance for {token_in} on binance")
        continue

    binance_balance[token_in] -= binance_amount_in
    binance_balance[token_out] += binance_amount_out

    binance_balance_min[token_in] = min(binance_balance_min[token_in], binance_balance[token_in])
    binance_balance_min[token_out] = min(binance_balance_min[token_out], binance_balance[token_out])

    if order.winning_surplus > order.spread_binance_order:
        # print(f"Order {order['cowOrder.id']} would lose auction")
        continue

    if onchain_balance[token_out] < binance_amount_out:
        print(f"Insufficient balance for {token_out} on chain")
        continue

    onchain_balance[token_out] -= binance_amount_out  # ! suppose we give all surplus to the maker
    onchain_balance[token_in] += binance_amount_in

    onchain_balance_min[token_in] = min(onchain_balance_min[token_in], onchain_balance[token_in])
    onchain_balance_min[token_out] = min(onchain_balance_min[token_out], onchain_balance[token_out])




print("Binance balance deltas")
for token, balance in binance_balance.items():
    print(f"\t{token}: {balance-starting_balance} (max-used: {starting_balance-binance_balance_min[token]})")
print("Onchain balance deltas")
for token, balance in onchain_balance.items():
    print(f"\t{token}: {balance-starting_balance} (max-used: {starting_balance-onchain_balance_min[token]})")

Binance balance deltas
	0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48: 641996.0422493871 (max-used: 111420.60580850404)
	0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: -306.8453140502097 (max-used: 489.61155425955076)
	0x6b175474e89094c44da98b954eedeac495271d0f: -182481.05936311116 (max-used: 182481.05936311116)
	0xdac17f958d2ee523a2206206994597c13d831ec7: 207.128328958177 (max-used: 50886.180356999976)
	0x2260fac5e5542a773aa44fbcfedf7c193bc2c599: 4.7413080965634435 (max-used: 0.11844384472351521)
Onchain balance deltas
	0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: 376.1785389074357 (max-used: 39.642645502695814)
	0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48: -818186.1901367395 (max-used: 972523.7435947396)
	0x6b175474e89094c44da98b954eedeac495271d0f: 25223.333819224034 (max-used: 0.0)
	0xdac17f958d2ee523a2206206994597c13d831ec7: 49949.79206799995 (max-used: 0.0)
